In [11]:
from typing import TypedDict, Annotated

from dotenv import load_dotenv

load_dotenv()

True

In [12]:
from langchain.chat_models import init_chat_model
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END

llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [13]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name."}
response = graph.invoke({ "messages": [message]})

In [14]:
response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name.', additional_kwargs={}, response_metadata={}, id='6cd69413-b060-4524-8f6e-818b8445e3e3'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--701db5bb-dd72-4407-b5b8-bbf83aedeb7a-0', usage_metadata={'input_tokens': 15, 'output_tokens': 3, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}})]

In [17]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break;
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state['messages'][-1].content)

Bot: Neil Armstrong
Bot: 1969


KeyboardInterrupt: Interrupted by user